## 1절. 뷰티풀솝과 파서

In [15]:
import requests
from requests_file import FileAdapter

s = requests.Session()
s.mount('file://', FileAdapter())
res = s.get('file:///sample.html')
res

<Response [200]>

In [16]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(res.content, 'html.parser')

In [17]:
el = soup.select_one('h1')
print(el)

<h1>Hello CSS</h1>


In [18]:
el.text, el.name, el.attrs

('Hello CSS', 'h1', {})

In [19]:
soup.select('h1, span')

[<h1>Hello CSS</h1>,
 <span>다른
 <b>요소가 반환</b></span>]

## 2절. requests를 이용한 웹 데이터 수집

## 3절. Selenium을 이용한 웹 데이터 수집

## 4절. 연습문제

In [37]:
# 실습형 1번
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

response = requests.get('https://www.yes24.com/24/category/bestseller')

if response.status_code == 200:
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    title_list = []

    # 선택자 copy result
    # #bestList > ol > li.num1 > p:nth-child(3) > a
    num_filter = re.compile('^num')

    num_tag = soup.find_all('li', class_=num_filter)
    for i in num_tag:
        selected_a_tag = i.select('p:nth-child(3) > a')
        title_list.append(selected_a_tag[0])
    
    title_list = [title.text for title in title_list]
    title_rank = [num for num in range(1, len(title_list) + 1)]
    
    title_df = pd.DataFrame(title_list, index=title_rank, columns=['영화제목'])
    # print(title_df)
else:
    print(response.status_code)

title_df.to_csv('bestseller.csv', encoding='utf-8-sig')

In [79]:
# 실습형 2번
response_news = requests.get('http://fs.jtbc.joins.com/RSS/economy.xml')

if response_news.status_code == 200:
    xml = response_news.content
    xml_soup = BeautifulSoup(xml, 'lxml')
    
    title_list = [title.text for title in xml_soup.select('item > title')]
    desc_list = [desc.text for desc in xml_soup.select('item > description')]
    date_list = [date.text for date in xml_soup.select('item > pubDate')]

    news_df = pd.DataFrame({
                            '제목' : title_list,
                            '내용' : desc_list,
                            '작성날짜' : date_list
                            })
else:
    print(response_news.status_code)

news_df.to_csv('news.txt', encoding='utf-8-sig')